In [2]:
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import leaguegamelog
from nba_api.stats.endpoints import boxscoreplayertrackv2
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import playergamelogs
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.static import teams
from json import JSONDecodeError
import pandas as pd
import requests
import time

In [3]:
def str_to_mins(inp):
    lst = inp.split(":")
    sec = int(lst[0])*60 + int(lst[1])
    return sec

In [4]:
games = pd.read_csv("../common-datasets/all_gamelogs.csv")
boxscores = pd.read_csv("../common-datasets/all_boxscores_concatenated.csv")
season_stats = pd.read_csv("../common-datasets/player-seasons_per_possession.csv")
playernames = pd.read_csv("relevant_players.csv")
starters_boxscores = boxscores.dropna(subset = ["START_POSITION"])
playernames = list(starters_boxscores["PLAYER_NAME"].unique())

In [5]:
len(playernames)

1541

In [218]:
lst = []
time_taken = [10]
weird_ids = []
start = timeit.default_timer()
verbose = True
for i in range(len(playernames)):
    time.sleep(0.36)
    player = playernames[i]
    successful = False
    decode_error = False
    while not successful:
        try:
            get_start = timeit.default_timer()
            pl = commonplayerinfo.CommonPlayerInfo(player, timeout=round(2*sum(time_taken)/len(time_taken), 0))
            get_end = timeit.default_timer()
            time_taken.append(get_end-get_start)
            successful = True
        except JSONDecodeError:
            weird_ids.append(player)
            decode_error = True
            break
        except:
            if verbose:
                print(f"Timeout error on iteration {i}, taking a 3 second break before resuming")
            time.sleep(3)
        
    if decode_error: continue
    players = pl.get_data_frames()[0]
    lst.append(players)
    if i%50 == 1:
        stop = timeit.default_timer()
        if verbose:
            print(f"{i-1}/{len(playernames)} done: {round(i/len(playernames)*100,1)}% ... has taken {round((stop-start)/60,0)} min, total projected: {round((stop-start)/(i/len(playernames))/60, 2)} min, remaining projected: {round(((stop-start)/(i/len(playernames))-(stop-start))/60, 2)} min")
boxscores_concat = pd.concat(lst, axis = 0, ignore_index = True)
boxscores_concat.to_csv(f"commonplayerinfo.csv",index=False)


0/2376 done: 0.0% ... has taken 0.0 min, total projected: 39.43 min, remaining projected: 39.42 min
50/2376 done: 2.1% ... has taken 0.0 min, total projected: 22.88 min, remaining projected: 22.39 min
100/2376 done: 4.3% ... has taken 1.0 min, total projected: 23.19 min, remaining projected: 22.2 min
Timeout error on iteration 147, taking a 3 second break before resuming
150/2376 done: 6.4% ... has taken 2.0 min, total projected: 24.21 min, remaining projected: 22.67 min
200/2376 done: 8.5% ... has taken 2.0 min, total projected: 24.12 min, remaining projected: 22.08 min
250/2376 done: 10.6% ... has taken 3.0 min, total projected: 24.29 min, remaining projected: 21.72 min
300/2376 done: 12.7% ... has taken 3.0 min, total projected: 24.22 min, remaining projected: 21.15 min
Timeout error on iteration 306, taking a 3 second break before resuming
400/2376 done: 16.9% ... has taken 4.0 min, total projected: 24.73 min, remaining projected: 20.56 min
Timeout error on iteration 440, taking a 

In [6]:
games.columns

Index(['Team_ID', 'GAME_ID', 'GAME_DATE', 'MATCHUP', 'WL', 'W', 'L', 'W_PCT',
       'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA',
       'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF',
       'PTS', 'SEASON', 'TEAM_ID'],
      dtype='object')

In [7]:
boxscores.columns

Index(['GAME_ID', 'TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_CITY', 'PLAYER_ID',
       'PLAYER_NAME', 'START_POSITION', 'COMMENT', 'MIN', 'SPD', 'DIST',
       'ORBC', 'DRBC', 'RBC', 'TCHS', 'SAST', 'FTAST', 'PASS', 'AST', 'CFGM',
       'CFGA', 'CFG_PCT', 'UFGM', 'UFGA', 'UFG_PCT', 'FG_PCT', 'DFGM', 'DFGA',
       'DFG_PCT'],
      dtype='object')

In [8]:
season_stats.columns

Index(['PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_ABBREVIATION', 'AGE', 'GP',
       'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'TOV',
       'STL', 'BLK', 'BLKA', 'PF', 'PFD', 'PTS', 'PLUS_MINUS',
       'NBA_FANTASY_PTS', 'DD2', 'TD3', 'GP_RANK', 'W_RANK', 'L_RANK',
       'W_PCT_RANK', 'MIN_RANK', 'FGM_RANK', 'FGA_RANK', 'FG_PCT_RANK',
       'FG3M_RANK', 'FG3A_RANK', 'FG3_PCT_RANK', 'FTM_RANK', 'FTA_RANK',
       'FT_PCT_RANK', 'OREB_RANK', 'DREB_RANK', 'REB_RANK', 'AST_RANK',
       'TOV_RANK', 'STL_RANK', 'BLK_RANK', 'BLKA_RANK', 'PF_RANK', 'PFD_RANK',
       'PTS_RANK', 'PLUS_MINUS_RANK', 'NBA_FANTASY_PTS_RANK', 'DD2_RANK',
       'TD3_RANK', 'CFID', 'CFPARAMS', 'SEASON'],
      dtype='object')

In [9]:
list_all_game_ids = list(games["GAME_ID"].unique())

In [17]:
game_info_dict = {k:{"home":dict(), "away":dict()} for k in list_all_game_ids}
for game_id in list_all_game_ids:
    relevant_games = games[(games['GAME_ID'] == game_id)]
    relevant_boxscores = boxscores[(boxscores['GAME_ID'] == game_id)]
    relevant_games.reset_index(drop=True, inplace=True)
    current_season = relevant_games.loc[0, "SEASON"]
    
   # second step.... add the appropriate team id to home vs away
    if "@" in relevant_games.loc[0, "MATCHUP"]:
        away_team_row = 0
        home_team_row = 1      
    else:
        away_team_row = 1
        home_team_row = 0
        
    game_info_dict[game_id]["winner"] = relevant_games.loc[0, "TEAM_ID"] if relevant_games.loc[0, "WL"] == "W" else relevant_games.loc[1, "TEAM_ID"]
    game_info_dict[game_id]["away"]["team_id"] = relevant_games.loc[away_team_row, "TEAM_ID"]
    away_team_id = relevant_games.loc[away_team_row, "TEAM_ID"]
    game_info_dict[game_id]["home"]["team_id"] = relevant_games.loc[home_team_row, "TEAM_ID"]
    home_team_id = relevant_games.loc[home_team_row, "TEAM_ID"]
    
    #log the win percentages
    game_info_dict[game_id]["away"]["W_PCT"] = relevant_games.loc[away_team_row, "W_PCT"]
    game_info_dict[game_id]["home"]["W_PCT"] = relevant_games.loc[home_team_row, "W_PCT"]
    
    #start logging the players
    game_info_dict[game_id]["home"]["players"] = dict()
    game_info_dict[game_id]["away"]["players"] = dict()
    for row in relevant_boxscores.itertuples():
        sec_played = str_to_mins(row.MIN)
        if row.START_POSITION in {"F","G","C"}:
            if row.TEAM_ID == home_team_id:
                game_info_dict[game_id]["home"]["players"][row.PLAYER_NAME] = {"sec_played":sec_played, "stats":dict()}
            else:
                game_info_dict[game_id]["away"]["players"][row.PLAYER_NAME] = {"sec_played":sec_played, "stats":dict()}
        

In [18]:
game_info_dict

{29701187: {'home': {'team_id': 1610612747,
   'W_PCT': nan,
   'players': {'Robert Horry': {'sec_played': 1581, 'stats': {}},
    'Rick Fox': {'sec_played': 921, 'stats': {}},
    "Shaquille O'Neal": {'sec_played': 2384, 'stats': {}},
    'Eddie Jones': {'sec_played': 2621, 'stats': {}},
    'Derek Fisher': {'sec_played': 1057, 'stats': {}}}},
  'away': {'team_id': 1610612762,
   'W_PCT': nan,
   'players': {'Karl Malone': {'sec_played': 1970, 'stats': {}},
    'Adam Keefe': {'sec_played': 692, 'stats': {}},
    'Greg Foster': {'sec_played': 1066, 'stats': {}},
    'Jeff Hornacek': {'sec_played': 1771, 'stats': {}},
    'John Stockton': {'sec_played': 1286, 'stats': {}}}},
  'winner': 1610612747},
 29701173: {'home': {'team_id': 1610612747,
   'W_PCT': nan,
   'players': {'Robert Horry': {'sec_played': 1996, 'stats': {}},
    'Rick Fox': {'sec_played': 1897, 'stats': {}},
    "Shaquille O'Neal": {'sec_played': 1970, 'stats': {}},
    'Eddie Jones': {'sec_played': 2147, 'stats': {}},
 